In [5]:
! pip install pypdf



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install langchain_community


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_community.document_loaders import PyPDFLoader


In [8]:
pip install -U langchain-text-splitters


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
loader=PyPDFLoader(r"C:\Users\ADMIN\Desktop\Data Sicence\innomatics internship\Gen-Application\keys\2404.07143.pdf")
pages=loader.load_and_split()


In [10]:
from langchain_text_splitters import NLTKTextSplitter
text_splitter = NLTKTextSplitter(chunk_size=500,chunk_overlap=100)
chunks=text_splitter.split_documents(pages)
print(len(chunks))
print(type(chunks[0]))


Created a chunk of size 568, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 633, which is longer than the specified 500


110
<class 'langchain_core.documents.base.Document'>


In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
chunks[0]


Document(page_content='Preprint.\n\nUnder review.\n\nLeave No Context Behind:\nEfficient Infinite Context Transformers with Infini-attention\nTsendsuren Munkhdalai, Manaal Faruqui and Siddharth Gopal\nGoogle\ntsendsuren@google.com\nAbstract\nThis work introduces an efficient method to scale Transformer-based Large\nLanguage Models (LLMs) to infinitely long inputs with bounded memory\nand computation.\n\nA key component in our proposed approach is a new at-\ntention technique dubbed Infini-attention.', metadata={'source': 'C:\\Users\\ADMIN\\Desktop\\Data Sicence\\innomatics internship\\Gen-Application\\keys\\2404.07143.pdf', 'page': 0})

In [13]:
pip install langchain-google-genai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
f = open(r"C:\Users\ADMIN\Desktop\Data Sicence\innomatics internship\Gen-Application\keys\.gemeniai.txt")
api_key = f.read()
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=api_key ,
                                               model="models/embedding-001")


In [1]:
pip install chromadb


  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/a4/e1/ce276f553811bd6c684cfe5f637a33ae6444750746f974a8f73d5dc92004/chromadb-0.5.0-py3-none-any.whl.metadata
  Using cached chromadb-0.5.0-py3-none-any.whl.metadata (7.3 kB)
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/e2/03/f3c8ba0a6b6e30d7d18c40faab90807c9bb5e9a1e3b2fe2008af624a9c97/build-1.2.1-py3-none-any.whl.metadata
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/ad/0f/feb7fd8957714498fc4a6be7f13408869619f868f418698a2d934afa82a7/fastapi-0.110.2-py3-none-any.whl.metadata
  Using cached fastapi-0.110.2-py3-none-any.whl.metadata (24 kB)
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/73/f5/cbb16fcbe277c1e0b8b3ddd188f2df0e0947f545c49119b589643632d156/uvicorn-0.29.0-py3


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import chromadb
import chromadb.config

In [15]:
# Store the chunks in vector store
from langchain_community.vectorstores import Chroma

# Embed each chunk and load it into the vector store
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")

# Persist the database on drive
db.persist()


In [16]:
# Setting a Connection with the ChromaDB
connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)


In [17]:
# Converting CHROMA db_connection to Retriever Object
retriever = connection.as_retriever(search_kwargs={"k": 5})

print(type(retriever))


<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [18]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [19]:
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot. 
    Your task is to provide assistance based on the context given by the user. 
    Make sure your answers are relevant and helpful."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context:
    {context}
    
    Question: 
    {question}
    
    Answer: """)
])


In [20]:
# loading the latest gemini-1.5-pro-latest
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(google_api_key=api_key, 
                                   model="gemini-1.5-pro-latest")


In [21]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()


In [23]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever |format_docs, "question":RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [25]:
response = rag_chain.invoke("What is a GenAI")
response

'## GenAI: Understanding the Term in Context\n\nUnfortunately, the provided text snippet doesn\'t directly define "GenAI." However, we can make some educated guesses based on the context:\n\n**Possible Interpretations of GenAI:**\n\n1. **Generative AI:** This is the most likely interpretation. The context discusses attention mechanisms, sparse transformers, and generating long sequences, all of which are closely related to generative AI models. These models are capable of creating new content like text, images, or code, similar to how humans do.\n\n2. **General AI:** While less likely, "GenAI" could potentially refer to artificial general intelligence (AGI). AGI refers to hypothetical AI with human-level cognitive abilities, capable of understanding, learning, and applying knowledge across various domains.\n\n3. **Specific AI Model or Technique:** There\'s a small chance "GenAI" refers to a specific AI model or technique mentioned elsewhere in the document but not included in this snip

In [26]:
from IPython.display import Markdown as md
md(response)

## GenAI: Understanding the Term in Context

Unfortunately, the provided text snippet doesn't directly define "GenAI." However, we can make some educated guesses based on the context:

**Possible Interpretations of GenAI:**

1. **Generative AI:** This is the most likely interpretation. The context discusses attention mechanisms, sparse transformers, and generating long sequences, all of which are closely related to generative AI models. These models are capable of creating new content like text, images, or code, similar to how humans do.

2. **General AI:** While less likely, "GenAI" could potentially refer to artificial general intelligence (AGI). AGI refers to hypothetical AI with human-level cognitive abilities, capable of understanding, learning, and applying knowledge across various domains.

3. **Specific AI Model or Technique:** There's a small chance "GenAI" refers to a specific AI model or technique mentioned elsewhere in the document but not included in this snippet. 

**To determine the precise meaning of GenAI, you would need to:**

* **Review the full document:** Look for a definition or explanation of "GenAI" within the text.
* **Consider the broader context:** Analyze the overall topic of the document and the specific section where "GenAI" is mentioned.
* **Consult external resources:** Search online for the term "GenAI" to see if it has a commonly accepted definition within the field of AI.

**Additional Clues from the Context:**

* The references to "sparse transformers" and "generating long sequences" suggest a focus on efficient and effective generation of content, which aligns well with the concept of generative AI. 
* The mention of "system-level optimization techniques" and specific hardware architecture further supports the idea of optimizing AI models for performance and resource efficiency. 
